In [148]:
!pip install pyworld
!pip install resampy


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 3.1/3.1 MB 639.2 kB/s eta 0:00:00
  Using cached setuptools-59.8.0-py3-none-any.whl (952 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.3.0
    Uninstalling setuptools-65.3.0:
      Successfully uninstalled setuptools-65.3.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.10.0 requires keras<2.11,>=2.10.0, but you have keras 2.11.0 which is incompatible.
tensorflow-gpu 2.10.0 requires tensorboard<2.11,>=2.10, but you have tensorboard 2.11.2 which is incompatible.
tensorflow-gpu 2.10.0 requires tensorflow-estimator<2.11,>=2.10.0, but you have tensorflow-estimator 2.11.0 which is incompatible.
ipykernel 6.11.0 requires setuptools>=60, but you have setuptools 59.8.0 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [156]:
import wave
import pyworld as pw
from scipy.io import wavfile
from scipy.io.wavfile import write

import resampy
from scipy import interpolate

import numpy as np

In [186]:
obj = wave.open("193326__margo_heston__wuh.wav","rb")

In [187]:
print(obj.getparams())

_wave_params(nchannels=1, sampwidth=2, framerate=44100, nframes=89523, comptype='NONE', compname='not compressed')


In [188]:
frames = obj.readframes(-1)
print(type(frames))
print(len(frames))

<class 'bytes'>
179046


In [237]:
rate , raw = wavfile.read("syllable.wav")
warr = np.array(raw)
print(len(warr))
print(warr)
mono = (warr).astype("double")
print(rate)

28224
[ -8   9  -2 ... -15 -11 -13]
44100


In [238]:
f0, sp, ap = pw.wav2world(mono, 44100.0)

In [239]:
brate , braw = wavfile.read("193326__margo_heston__wuh.wav")
bwarr = np.array(braw)
print(len(bwarr))
bmono = (bwarr).astype("double")
print(bmono)

456917
[-1783. -1834. -1966. ...    -3.    -3.    -3.]


In [240]:
print(brate)
print(rate)

192000
44100


In [278]:
bf0, bsp, bap = pw.wav2world(bmono, 44100.0)

In [279]:
kernel = np.hamming(30)
print(len(bf0))
bf0 = resampy.resample(bf0, len(bf0), len(f0))
bsmooth= np.convolve(bf0, kernel,mode = 'same')
print(bsmooth)

2073
[ 412.92808388  439.74750291  457.473361    466.66910661  465.69564464
  455.63608671  446.34929205  426.60155715  400.05043113  375.06091798
  352.021982    334.8931306   325.34448695  327.35088869  336.72381587
  361.57042844  399.76477119  439.44798493  489.63780261  550.62863655
  614.57724626  676.93313859  740.24539266  802.06332099  858.23159835
  908.50359646  952.5026246   989.99602409 1021.61472348 1047.71860783
 1069.27499849 1087.49888816 1101.40841158 1108.97796058 1115.96384552
 1121.77225844 1120.32792196 1119.4018737  1116.97489452 1110.43237461
 1100.64079086 1089.19120416 1075.34751254 1060.49784623 1048.5222275
 1032.38480281 1015.56797198 1007.04508186 1004.3245807   997.73096024
  994.54875839  993.12135546  989.2202881   988.09984064  986.18281696
  988.28986154  989.28790102  986.11414733  980.99013516  972.34886344
  962.81691422  950.3287572   940.13459975  935.31996738  933.97851661
  937.45221361  941.83634831  951.19988035  967.2400058   987.20709169
 1

In [301]:

#b_norm = (bsmooth-np.min(bsmooth))/(np.max(bsmooth)-np.min(bsmooth))
b_ts = bsmooth * np.average(f0)/np.average(bsmooth)
print(np.average(f0))
print(np.average(bf0))
print(np.average(b_ts))
#b_ts = resampy.resample(b_ts, len(b_norm), len(f0))
print(len(b_ts))

#fsmooth = b_ts*0.8 + f0*0.8
fsmooth = f0*0.9
print(np.average(fsmooth))

196.93405090544394
59.16922358193157
196.93405090544394
129
177.24064581489958


In [302]:
y = pw.synthesize(fsmooth, sp, ap, 44100.0).astype("int32")

In [303]:
print(y)
scaled = np.int16((y/np.max(np.abs(y))) * 32767)
print(len(scaled))
print(len(warr))

[0 0 0 ... 6 3 3]
28444
28224


In [304]:
wavfile.write('test.wav', rate, scaled)